In [ ]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizate pentru a reduce dimensiunea la una prestabilita, util cand marimea input ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 49, 49])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 24, 24]
*   [1, 10, 9, 9]
*  [1, 3, 2, 2]



In [ ]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale
#1
layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(30,30), stride=(3,3))
print("Conv1 result shape",layer(dummy_input_tensor).shape)
#2
layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(60,60), stride=(5,5))
print("Conv2 result shape",layer(dummy_input_tensor).shape)
#3
layer = nn.AdaptiveAvgPool2d(output_size=(2,2))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 24, 24])
Conv2 result shape torch.Size([1, 10, 9, 9])
Global Pool result shape torch.Size([1, 3, 2, 2])


## Instantierea seturilor de date

In [ ]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

Files already downloaded and verified


## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __preproc_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *preproc_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [ ]:
import torch
import numpy as np
##
import torch.utils.data as data 
##
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize

def preproc_fn(examples):
  ### Completati codul pentru cerinta aici
  processed_images = []
  processed_labels = []

  for example in examples:
    tensor_image = to_tensor(example[0])
    normalized_tensor_image = normalize(tensor_image, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])

    processed_images.append(normalized_tensor_image)
    
    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)

  return torch_images, torch_labels
  return examples


train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=preproc_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=preproc_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii de 2 ori
    * Liniarizati iesirea din cel de-al doilea strat convolutional
    * Adaugat stratul final de tipul 'fully-connected'
    * Folositi o functie de activare la alegere

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(3,3), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(3,3), stride=(2,2))

    self.linear1 = nn.Linear(16*7*7, 120)
    self.linear2 = nn.Linear(120, 84)
    self.out = nn.Linear(84,10)

    self.activation1 = nn.Tanh()

  def forward(self,x):
    x = self.conv1(x)
    x = self.activation1(x)

    x = self.conv2(x)
    x = self.activation1(x)

    x = x.reshape(x.shape[0], -1)
    
    x = self.linear1(x)
    x = self.activation1(x)

    x = self.linear2(x)
    x = self.activation1(x)

    out = self.out(x)
    out = self.activation1(out)

    return out
    """
  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(3, 6, 3)
    self.conv2 = nn.Conv2d(6, 16, 3)

    self.fc1 = nn.Linear(16 * 6 * 6, 120)  
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), 2)

    x = F.max_pool2d(F.relu(self.conv2(x)), 2)

    x = x.view(-1, self.num_flat_features(x))

    x = F.relu(self.fc1(x))

    x = F.relu(self.fc2(x))

    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
      num_features *= s
    return num_features    
    """

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Numarul de epoci
  * Retea
  * Optimizator
  * Alegeti functia de cost

In [ ]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
network = Net()

# Definiti optimizatorul
optimizer = optim.SGD(network.parameters(), lr=1e-2)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
optimizer.zero_grad()


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()


## Definirea functiei de antrenare

In [ ]:
def train_fn(epochs: int, train_loader: data.DataLoader, test_loader: data.DataLoader, 
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  # Iteram prin numarul de epoci
  for e in range(epochs):
    # Iteram prin fiecare exemplu din dataset
    for images, labels in train_loader:

      # Aplicam reteaua neurala pe imaginile de intrare
      images = np.reshape(images, (500,3,32,32))
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe adnotarile imaginilor 
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a aplica gradientii pe parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()
    
    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Caluculul acuratetii
    count = len(test_loader)
    correct = 0

    for test_image, test_label in test_loader:
      test_image = np.reshape(test_image, (1,3,32,32))
      out_class = torch.argmax(net(test_image))
      if out_class == test_label:
        correct += 1

    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e, (correct / count) * 100))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [ ]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.289318084716797
Acuratetea la finalul epocii 0 este 16.47%
Loss-ul la finalul epocii 1 are valoarea 2.245211362838745
Acuratetea la finalul epocii 1 este 19.99%
Loss-ul la finalul epocii 2 are valoarea 2.1588380336761475
Acuratetea la finalul epocii 2 este 23.53%
Loss-ul la finalul epocii 3 are valoarea 2.1145517826080322
Acuratetea la finalul epocii 3 este 25.22%
Loss-ul la finalul epocii 4 are valoarea 2.1176843643188477
Acuratetea la finalul epocii 4 este 26.44%
Loss-ul la finalul epocii 5 are valoarea 2.071138620376587
Acuratetea la finalul epocii 5 este 27.46%
Loss-ul la finalul epocii 6 are valoarea 2.063682794570923
Acuratetea la finalul epocii 6 este 28.34%
Loss-ul la finalul epocii 7 are valoarea 2.026707410812378
Acuratetea la finalul epocii 7 este 28.79%
Loss-ul la finalul epocii 8 are valoarea 2.002866268157959
Acuratetea la finalul epocii 8 este 29.19%
Loss-ul la finalul epocii 9 are valoarea 2.0210671424865723
Acuratetea la final

## Reteaua LeNet

### Cerinte
  * **(3p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o


![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet


In [ ]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.layer1 = nn.Conv2d(in_channels=3, out_channels= 6, kernel_size=(5,5), stride=(1,1))
    self.layer2 = nn.AvgPool2d(kernel_size = 2, stride = 2)
    self.layer3 = nn.Conv2d(in_channels=6, out_channels= 16, kernel_size=(5,5), stride=(1,1))
    self.layer4 = nn.AvgPool2d(kernel_size = 2, stride = 2)
    self.layer5 = nn.Conv2d(in_channels=16, out_channels= 120, kernel_size=(5,5), stride=(1,1))
    self.layer6 = nn.Linear(120,84)
    self.output = nn.Linear(84,10)
    
    self.activation1 = nn.Tanh()
    self.activation2 = nn.Softmax()


  def forward(self,x):
    x = self.layer1(x)
    x = self.activation1(x)

    x = self.layer2(x)
    x = self.activation1(x)

    x = self.layer3(x)
    x = self.activation1(x)

    x = self.layer4(x)
    x = self.activation1(x)

    x = self.layer5(x)
    x = self.activation1(x)

    x = x.reshape(x.shape[0], -1)
    
    x = self.layer6(x)
    x = self.activation1(x)

    x = self.output(x)
    x = self.activation2(x)
    return x

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * Redefiniti obiectele pentru a antrena reteaua LeNet

In [ ]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
lenet = LeNet()

# Definiti optimizatorul
lenet_optimizer = optimizer = optim.SGD(network.parameters(), lr=1e-2)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()


## Antrenarea retelei LeNet

In [ ]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Loss-ul la finalul epocii 0 are valoarea 2.302382230758667
Acuratetea la finalul epocii 0 este 10.88%
Loss-ul la finalul epocii 1 are valoarea 2.303309679031372
Acuratetea la finalul epocii 1 este 10.88%
Loss-ul la finalul epocii 2 are valoarea 2.3019018173217773
Acuratetea la finalul epocii 2 este 10.88%
Loss-ul la finalul epocii 3 are valoarea 2.303027391433716
Acuratetea la finalul epocii 3 este 10.88%
Loss-ul la finalul epocii 4 are valoarea 2.302363872528076
Acuratetea la finalul epocii 4 este 10.88%
Loss-ul la finalul epocii 5 are valoarea 2.302241563796997
Acuratetea la finalul epocii 5 este 10.88%
Loss-ul la finalul epocii 6 are valoarea 2.302755355834961
Acuratetea la finalul epocii 6 este 10.88%
Loss-ul la finalul epocii 7 are valoarea 2.3027420043945312
Acuratetea la finalul epocii 7 este 10.88%
Loss-ul la finalul epocii 8 are valoarea 2.3024978637695312
Acuratetea la finalul epocii 8 este 10.88%
Loss-ul la finalul epocii 9 are valoarea 2.302110433578491
Acuratetea la finalu

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* torch.nn.BatchNorm2d(num_features)
* torch.nn.InstanceNorm2d(num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de functii de activate:


* Relu
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu aceste elemente in cadrul retelei LeNet definita mai devreme, pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  
